## NL to BI Architecture. 

Looker symantic layer can be a good helper to convert natural language to SQL and show a dashboard dependent on the given query. 


In [1]:
#!pip install looker-sdk
#!pip install google-cloud-aiplatform langchain

In [19]:
question = "2022년 생성일 기준으로, 금융업에 속한 국민연금 평균 납부액을 월별로 보여주세요."
#question = "As of the creation date in 2022, please show me the monthly average national pension data for the financial industry along with data from three months ago"

In [20]:
import looker_sdk

sdk = looker_sdk.init31()  # or init40() for the older v3.1 API
my_user = sdk.me()

In [21]:
def listup_explore(model_name):
  
  return sdk.all_projects()

In [22]:
from pprint import pprint

for lookml_model in sdk.all_lookml_models():
  print(lookml_model.name)
  for nav_explore in lookml_model.explores:
    print(nav_explore)

sample_project
lookml_hol_sample
LookmlModelNavExplore(name='postal_location', description=None, label='Postal Location', hidden=False, group_label='Lookml Hol Sample', can={})
LookmlModelNavExplore(name='bizcategory', description='이 View는 한국 산업 코드 정보를 담고 있습니다. 대분류(Catl1)부터 세분류(CatL4)까지 가지고 있으며, 산업코드를 구분짓는데 사용합니다. ', label='Bizcategory', hidden=False, group_label='Lookml Hol Sample', can={})
LookmlModelNavExplore(name='national_pension_category_summary', description='이 화면은 국민연금에 대한 산업별 월 단위 통계 정보를 제공합니다. 제공되는 정보는 다음과 같습니다.\n평균/총 근로자 수\n평균/총 연금 납부액\n평균/총 신규 근로자 수\n평균/총 실업 근로자 수\n또한, 이 화면은 월별 통계를 비교할 수 있는 추가적인 기능도 제공합니다.', label='National Pension Category Summary', hidden=False, group_label='Lookml Hol Sample', can={})
LookmlModelNavExplore(name='national_pension_mom', description="This view provides monthly company-specific payment details for the National Pension Service. It includes information on the number of employees who contribute to the company's pension, and the amount of pensi

In [23]:
from vector_util import VectorDatabase

vdb = VectorDatabase()

vdb.truncate_table()

In [24]:
from langchain.embeddings import VertexAIEmbeddings

embeddings = VertexAIEmbeddings()

In [25]:

def write_navexplore_to_vdb(lookml_model):
  for nav_explore in lookml_model.explores:
    if(nav_explore.description is None):
      continue 
    description = nav_explore.description
    desc_vector = embeddings.embed_query(description)
    vdb.insert_record(sql=None, parameters=None, description=description, explore_view=nav_explore.name, model_name=lookml_model.name, table_name=None, column_schema=None, desc_vector=desc_vector)

for lookml_model in sdk.all_lookml_models():
  write_navexplore_to_vdb(lookml_model)

In [26]:

def get_right_explore(question):
  test_embedding =  embeddings.embed_query(question)
  with vdb.get_connection() as conn:
    try:
      with conn.cursor() as cur:
        select_record = (str(test_embedding).replace(' ',''),)
        cur.execute(f"with tb_settings as (select %s::vector as compared_vec) SELECT model_name, explore_view, description, 1 - cosine_distance(desc_vector,compared_vec) as cosine_sim FROM rag_test, tb_settings where (1 - cosine_distance(desc_vector,compared_vec)) > 0.5 order by 4 desc limit 4", select_record)
        result = cur.fetchone()
        print(result[0], result[1], result[2], result[3])
        return result[0], result[1]
    except Exception as e:
      print(e)
  return None, None

lookml_model, lookml_explore = get_right_explore(question)  

lookml_hol_sample national_pension_category_summary 이 화면은 국민연금에 대한 산업별 월 단위 통계 정보를 제공합니다. 제공되는 정보는 다음과 같습니다.
평균/총 근로자 수
평균/총 연금 납부액
평균/총 신규 근로자 수
평균/총 실업 근로자 수
또한, 이 화면은 월별 통계를 비교할 수 있는 추가적인 기능도 제공합니다. 0.6933283122159007


In [27]:
def get_field_values(model_name, explore_name):

  # API Call to pull in metadata about fields in a particular explore
  explore = sdk.lookml_model_explore(
    lookml_model_name=model_name,
    explore_name=explore_name,
    fields="id, name, description, fields",
  )

  my_fields = []

  # Iterate through the field definitions and pull in the description, sql,
  # and other looker tags you might want to include in  your data dictionary.
  if explore.fields and explore.fields.dimensions:
    for dimension in explore.fields.dimensions:
      if dimension.hidden:
        #print(dimension)
        continue
      dim_def = {
        "field_type": "Dimension",
        "view_name": dimension.view_label,
        "field_name": dimension.name,
        "type": dimension.type,
        "description": dimension.description,
        #"sql": dimension.sql,
      }
      my_fields.append(dim_def)
  if explore.fields and explore.fields.measures:
    for measure in explore.fields.measures:
      mes_def = {
        "field_type": "Measure",
        "view_name": measure.view_label,
        "field_name": measure.name,
        "type": measure.type,
        "description": measure.description,
        #"sql": measure.sql,
      }
      my_fields.append(mes_def)
  if explore.fields and explore.fields.parameters:
    for parameter in explore.fields.parameters:
      par_def = {
        "field_type": "Parameter",
        "view_name": parameter.view_label,
        "field_name": parameter.name,
        "default_filter_value": parameter.default_filter_value,
        "type": parameter.type,
        "description": parameter.description,
        #"sql": parameter.sql,
      }
      my_fields.append(par_def)
  return my_fields

In [28]:
# for lookml_model in sdk.all_lookml_models():
#   print('lookml model name:' + lookml_model.name)
#   for one_explore in lookml_model.explores:
#     print('explore name:' + one_explore.name)
#     pprint(get_field_values(lookml_model.name, one_explore.name))
fields = get_field_values(lookml_model, lookml_explore)

In [29]:
fields

[{'field_type': 'Dimension',
  'view_name': 'Bizcategory',
  'field_name': 'bizcategory.cat_code',
  'type': 'string',
  'description': None},
 {'field_type': 'Dimension',
  'view_name': 'Bizcategory',
  'field_name': 'bizcategory.cat_l1',
  'type': 'string',
  'description': 'In general, largest catgories in Korea Industry segment table such like 건설업, 제조업'},
 {'field_type': 'Dimension',
  'view_name': 'Bizcategory',
  'field_name': 'bizcategory.cat_l1_code',
  'type': 'string',
  'description': None},
 {'field_type': 'Dimension',
  'view_name': 'Bizcategory',
  'field_name': 'bizcategory.cat_l2',
  'type': 'string',
  'description': None},
 {'field_type': 'Dimension',
  'view_name': 'Bizcategory',
  'field_name': 'bizcategory.cat_l2_code',
  'type': 'string',
  'description': None},
 {'field_type': 'Dimension',
  'view_name': 'Bizcategory',
  'field_name': 'bizcategory.cat_l3',
  'type': 'string',
  'description': None},
 {'field_type': 'Dimension',
  'view_name': 'Bizcategory',
  'fi

In [30]:
import vertexai
from langchain.chat_models import ChatVertexAI
from langchain.llms import VertexAI
import os

PROJECT_ID = os.getenv("PROJECT_ID")  # @param {type:"string"}
vertexai.init(project=PROJECT_ID, location="us-central1")

llm_vertex = VertexAI(
    #model_name="text-bison@latest",
    #model_name="text-unicorn@001",
    model_name="text-bison-32k",
    max_output_tokens=1020,
    temperature=0,
    top_p=0.8,
    top_k=40,
)


In [31]:
llm = llm_vertex

In [32]:
def choose_right_fields(fields, question):
  sample_json = """
{
  "dimensions": [
    "dimension1",
  ],
  "measures": [
    "measure1",
  ],
  "filters": [
    {
      "field_name": "field_name1",
      "values": [
        "value1"
      ]
    }
  ],
  "sorts": [
    {
      "field_name": "field_name1",
      "direction": "asc"
    }
  ],
  "parameters": [
    "param1",
  ],
  "pivots": [
    "field1"
  ],
  "hidden_fields": [
    "field1"
  ]
}
  """

  prompt_template = """As a looker developer, choose right dimesions and measures for the question below. 
  You should choose right fields as least as possible and sort fields must be choosen in the dimension fields.

  fields : {fields}

  question: {question}

  answer format: json
  {sample_json}
  """
  response = llm.predict(prompt_template.format(fields=fields, question=question, sample_json=sample_json))
  return response




In [33]:
# Test Code.
# get_field_values(lookml_model, lookml_explore)


In [34]:
schema_fields = get_field_values(lookml_model, lookml_explore)
response = choose_right_fields(fields=schema_fields, question=question) #S


In [35]:
response

' ```json\n{\n  "dimensions": [\n    "national_pension_category_summary.data_create_yearmonth_year",\n    "national_pension_category_summary.data_create_yearmonth_month",\n    "bizcategory.cat_l1"\n  ],\n  "measures": [\n    "national_pension_category_summary.avg_of_monthly_fixed_amount"\n  ],\n  "filters": [\n    {\n      "field_name": "bizcategory.cat_l1",\n      "values": [\n        "금융업"\n      ]\n    },\n    {\n      "field_name": "national_pension_category_summary.data_create_yearmonth_year",\n      "values": [\n        "2022"\n      ]\n    }\n  ],\n  "sorts": [\n    {\n      "field_name": "national_pension_category_summary.data_create_yearmonth_year",\n      "direction": "asc"\n    },\n    {\n      "field_name": "national_pension_category_summary.data_create_yearmonth_month",\n      "direction": "asc"\n    }\n  ],\n  "parameters": [],\n  "pivots": [],\n  "hidden_fields": []\n}\n```'

In [36]:
import json

def parse_json_response(llm_json_response) -> any:
  #print('llm response:'+ response)
  start_char = '['
  end_char = ']'
  if llm_json_response.find('[') == -1 or llm_json_response.find('{') < llm_json_response.find('[') :
    start_char = '{'
    end_char = '}'
  start_index = llm_json_response.find(start_char)
  end_index = llm_json_response.rfind(end_char)
  json_data = llm_json_response[start_index:end_index+1]
  parsed_json = json.loads(json_data)
  return parsed_json


In [37]:
import ast

def parse_python_object(llm_python_object) -> any:
  print('llm response:'+ llm_python_object)
  if llm_python_object.find('{') == -1:
    start_char = '['
    end_char = ']'
  elif llm_python_object.find('[') == -1 or llm_python_object.find('{') < llm_python_object.find('[') :
    start_char = '{'
    end_char = '}'
  start_index = llm_python_object.find(start_char)
  end_index = llm_python_object.rfind(end_char)
  object_data = llm_python_object[start_index:end_index+1]
  print(object_data)
  parsed_object = ast.literal_eval(object_data)
  return parsed_object


In [38]:
response_object = parse_json_response(response)

In [39]:
response_object

{'dimensions': ['national_pension_category_summary.data_create_yearmonth_year',
  'national_pension_category_summary.data_create_yearmonth_month',
  'bizcategory.cat_l1'],
 'measures': ['national_pension_category_summary.avg_of_monthly_fixed_amount'],
 'filters': [{'field_name': 'bizcategory.cat_l1', 'values': ['금융업']},
  {'field_name': 'national_pension_category_summary.data_create_yearmonth_year',
   'values': ['2022']}],
 'sorts': [{'field_name': 'national_pension_category_summary.data_create_yearmonth_year',
   'direction': 'asc'},
  {'field_name': 'national_pension_category_summary.data_create_yearmonth_month',
   'direction': 'asc'}],
 'parameters': [],
 'pivots': [],
 'hidden_fields': []}

In [40]:
def get_field_type(field_name, schema_fields):
  for field in schema_fields:
    if field['field_name'] == field_name:
      return field['type']

In [41]:
# from langchain.output_parsers import PydanticOutputParser
# from langchain.pydantic_v1 import BaseModel, Field, validator
# from typing import List

# class LookerFilterValue(BaseModel):
#   field_name: str = Field(description="field_name")
#   values: List[str] = Field(description="values")

# class LookerSortField(BaseModel):
#   field_name: str = Field(description="field_name")
#   direction: str = Field(description="direction")

# class LookerQueryParameters(BaseModel):
#   dimensions: List[str] = Field(description="dimensions")
#   measures: List[str] = Field(description="measures")
#   parameters: List[str] = Field(description="parameters")
#   filters: List[LookerFilterValue] = Field(description="filters")
#   sorts: List[LookerSortField] = Field(description="sorts")
#   pivots: List[str] = Field(description="pivot - These fields are used as pivots in the chart.")
#   hidden_fields: List[str] = Field(description="hidden_fields - These fields are used as filters but are not shown in the chart.")

# def get_right_fields_object(response):
#   parser = PydanticOutputParser(pydantic_object=LookerQueryParameters)
#   return parser.parse(response)


In [42]:
#related_fields = get_right_fields_object(response)
related_fields = parse_json_response(response)

In [43]:
related_fields['filters']

[{'field_name': 'bizcategory.cat_l1', 'values': ['금융업']},
 {'field_name': 'national_pension_category_summary.data_create_yearmonth_year',
  'values': ['2022']}]

In [44]:
def decide_to_retrieve_values_for_the_filters(related_fields):
  output_sample = """
  {
    "required_target": ["field1","field2"]
  }
  """
  prompt_template = """As a looker developer, decide whether to retrieve values for the filters below. 
  For example, date / timestamp columns don't need to retrieve values. but string columns need to retrieve values from the database.

  filter fields : {filter_fields}

  output sample : json array
  {output_sample}
  """
  response = llm.predict(prompt_template.format(filter_fields=related_fields['filters'], output_sample=output_sample))
  return response

In [45]:
retrieve_target_filters = decide_to_retrieve_values_for_the_filters(related_fields)

In [46]:
retrieve_target_filters

' ```json\n  {\n    "required_target": ["bizcategory.cat_l1"]\n  }\n  ```'

In [47]:

# class LookerFilterRetrieves(BaseModel):
#   required_target: List[str] = Field(description="required_target")

# def get_retrieve_target_filters_object(retrieve_target_filters):
#   parser = PydanticOutputParser(pydantic_object=LookerFilterRetrieves)
#   return parser.parse(retrieve_target_filters)


In [48]:
retrieve_target_filter_obj = parse_json_response(retrieve_target_filters)

In [49]:
def get_user_input_value_for_filter_field(related_fields, field_name):
  for filter in related_fields['filters']:
    if filter['field_name'] == field_name:
      return filter['values']
  return ""


In [50]:
# retrieve_target_filter_obj = get_retrieve_target_filters_object(retrieve_target_filters)

In [51]:
retrieve_target_filter_obj

{'required_target': ['bizcategory.cat_l1']}

In [52]:
from looker_sdk.sdk.api31 import models as ml

def get_validated_filter_values_from_looker(lookml_model, lookml_explore, retrieve_target_filter_obj):
  choose_right_filter_value_list = []
  for retrieve_target_filter in retrieve_target_filter_obj['required_target']:
    print(retrieve_target_filter)
    query_template = ml.WriteQuery(model=lookml_model, view=lookml_explore,fields=[retrieve_target_filter])
    query = sdk.create_query(query_template)
    json_ = sdk.run_query(query.id, "json")
    print(json_)
    choose_right_filter_value_list.append({ retrieve_target_filter : json_})
  return choose_right_filter_value_list
    


In [53]:
retrieve_filter_and_values = get_validated_filter_values_from_looker(lookml_model, lookml_explore, retrieve_target_filter_obj)

bizcategory.cat_l1
[{"bizcategory.cat_l1":"가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동"},
{"bizcategory.cat_l1":"건 설 업"},
{"bizcategory.cat_l1":"공공 행정, 국방 및 사회보장 행정"},
{"bizcategory.cat_l1":"광업"},
{"bizcategory.cat_l1":"교육서비스업"},
{"bizcategory.cat_l1":"금융 및 보험업"},
{"bizcategory.cat_l1":"도매 및 소매업"},
{"bizcategory.cat_l1":"보건업 및 사회복지 서비스업"},
{"bizcategory.cat_l1":"부동산업"},
{"bizcategory.cat_l1":"사업시설 관리, 사업 지원 및 임대 서비스업"},
{"bizcategory.cat_l1":"수도, 하수 및 폐기물 처리, 원료 재생업"},
{"bizcategory.cat_l1":"숙박 및 음식점업"},
{"bizcategory.cat_l1":"예술, 스포츠 및 여가관련 서비스업"},
{"bizcategory.cat_l1":"운수 및 창고업"},
{"bizcategory.cat_l1":"전기, 가스, 증기 및 공기 조절 공급업"},
{"bizcategory.cat_l1":"전문, 과학 및 기술 서비스업"},
{"bizcategory.cat_l1":"정보통신업"},
{"bizcategory.cat_l1":"제조업"},
{"bizcategory.cat_l1":"협회 및 단체, 수리 및 기타 개인 서비스업"},
{"bizcategory.cat_l1":"협회 및 단체, 수리 및 기타 개인서비스"}]


In [54]:
retrieve_filter_and_values

[{'bizcategory.cat_l1': '[{"bizcategory.cat_l1":"가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동"},\n{"bizcategory.cat_l1":"건 설 업"},\n{"bizcategory.cat_l1":"공공 행정, 국방 및 사회보장 행정"},\n{"bizcategory.cat_l1":"광업"},\n{"bizcategory.cat_l1":"교육서비스업"},\n{"bizcategory.cat_l1":"금융 및 보험업"},\n{"bizcategory.cat_l1":"도매 및 소매업"},\n{"bizcategory.cat_l1":"보건업 및 사회복지 서비스업"},\n{"bizcategory.cat_l1":"부동산업"},\n{"bizcategory.cat_l1":"사업시설 관리, 사업 지원 및 임대 서비스업"},\n{"bizcategory.cat_l1":"수도, 하수 및 폐기물 처리, 원료 재생업"},\n{"bizcategory.cat_l1":"숙박 및 음식점업"},\n{"bizcategory.cat_l1":"예술, 스포츠 및 여가관련 서비스업"},\n{"bizcategory.cat_l1":"운수 및 창고업"},\n{"bizcategory.cat_l1":"전기, 가스, 증기 및 공기 조절 공급업"},\n{"bizcategory.cat_l1":"전문, 과학 및 기술 서비스업"},\n{"bizcategory.cat_l1":"정보통신업"},\n{"bizcategory.cat_l1":"제조업"},\n{"bizcategory.cat_l1":"협회 및 단체, 수리 및 기타 개인 서비스업"},\n{"bizcategory.cat_l1":"협회 및 단체, 수리 및 기타 개인서비스"}]'}]

In [55]:
def choose_right_filter_value(filter_values, wanted_value):
  example_json = "[ 'value1' , 'value2' , 'value3' ]"
  prompt_template = """As a looker developer, choose right filter value for the wanted value below without changing filter value itself.

  output format : json array
  {example_json}

  filter_values : {filter_values}

  wanted_values: {wanted_value}

  """
  response = llm.predict(prompt_template.format(example_json=example_json,filter_values=filter_values, wanted_value=wanted_value))
  return response  

In [56]:
def get_appropriate_filter_value_pair(related_fields, retrieve_filter_and_values):
  filter_value_pair = []
  for filter_and_values in retrieve_filter_and_values:
    field_name = list(filter_and_values.keys())[0]
    print(filter_and_values)
    user_input_value = get_user_input_value_for_filter_field(related_fields, field_name)
    print(user_input_value)
    actual_value = choose_right_filter_value(filter_and_values, user_input_value)
    print(actual_value)  
    value_object = parse_python_object(actual_value)
    print(value_object)
    filter_value_pair.append({ field_name : value_object})
  return filter_value_pair


In [57]:
valid_filter_values = get_appropriate_filter_value_pair(related_fields, retrieve_filter_and_values)

{'bizcategory.cat_l1': '[{"bizcategory.cat_l1":"가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동"},\n{"bizcategory.cat_l1":"건 설 업"},\n{"bizcategory.cat_l1":"공공 행정, 국방 및 사회보장 행정"},\n{"bizcategory.cat_l1":"광업"},\n{"bizcategory.cat_l1":"교육서비스업"},\n{"bizcategory.cat_l1":"금융 및 보험업"},\n{"bizcategory.cat_l1":"도매 및 소매업"},\n{"bizcategory.cat_l1":"보건업 및 사회복지 서비스업"},\n{"bizcategory.cat_l1":"부동산업"},\n{"bizcategory.cat_l1":"사업시설 관리, 사업 지원 및 임대 서비스업"},\n{"bizcategory.cat_l1":"수도, 하수 및 폐기물 처리, 원료 재생업"},\n{"bizcategory.cat_l1":"숙박 및 음식점업"},\n{"bizcategory.cat_l1":"예술, 스포츠 및 여가관련 서비스업"},\n{"bizcategory.cat_l1":"운수 및 창고업"},\n{"bizcategory.cat_l1":"전기, 가스, 증기 및 공기 조절 공급업"},\n{"bizcategory.cat_l1":"전문, 과학 및 기술 서비스업"},\n{"bizcategory.cat_l1":"정보통신업"},\n{"bizcategory.cat_l1":"제조업"},\n{"bizcategory.cat_l1":"협회 및 단체, 수리 및 기타 개인 서비스업"},\n{"bizcategory.cat_l1":"협회 및 단체, 수리 및 기타 개인서비스"}]'}
['금융업']
 [
    "금융 및 보험업"
   ]
llm response: [
    "금융 및 보험업"
   ]
[
    "금융 및 보험업"
   ]
['금융 및 보험업']


In [58]:
valid_filter_values

[{'bizcategory.cat_l1': ['금융 및 보험업']}]

In [80]:
# test_values = [{
# 'bizcategory.cat_l1': ["금융 및 보험업", "건 설 업"]
# }]

# for filter_values in test_values:
#   print(filter_values)
#   filter_name = list(filter_values.keys())[0]
#   print(filter_name)
#   for filter_value in list(filter_values.values())[0]:
#     print(filter_value)

In [60]:
def get_quoted_value(valid_filter_values, field_name):
  values = []
  for filter_values in valid_filter_values:
    field_name_cmp = list(filter_values.keys())[0]
    if field_name_cmp == field_name:
      for field_value in list(filter_values.values())[0]:
        field_type = get_field_type(field_name, schema_fields)
        if field_type == 'string':
          values.append(field_value)
        else:
          values.append(str(field_value))
  return values
  

In [61]:
from typing import cast, Dict, List, Union

# LookML Filter values MUST be a string. 
def get_lookml_filter_values(valid_filter_values, filters):
  filter_dict:Dict[str, Any] = {}
  for filter in filters:
    field_name = filter['field_name']
    quoted_values = get_quoted_value(valid_filter_values, field_name)
    if(quoted_values == []):
      print(filter['values'])
      filter_dict[field_name] = ",".join(filter['values'])
      continue
    filter['values'] = quoted_values
    filter_dict[field_name] = ",".join(quoted_values)
  return filter_dict

In [62]:
print(get_lookml_filter_values(valid_filter_values, related_fields['filters']))

['2022']
{'bizcategory.cat_l1': '금융 및 보험업', 'national_pension_category_summary.data_create_yearmonth_year': '2022'}


In [63]:
for sorted_column in related_fields['sorts']:
  print(sorted_column)

{'field_name': 'national_pension_category_summary.data_create_yearmonth_year', 'direction': 'asc'}
{'field_name': 'national_pension_category_summary.data_create_yearmonth_month', 'direction': 'asc'}


In [64]:
print(related_fields)

{'dimensions': ['national_pension_category_summary.data_create_yearmonth_year', 'national_pension_category_summary.data_create_yearmonth_month', 'bizcategory.cat_l1'], 'measures': ['national_pension_category_summary.avg_of_monthly_fixed_amount'], 'filters': [{'field_name': 'bizcategory.cat_l1', 'values': ['금융 및 보험업']}, {'field_name': 'national_pension_category_summary.data_create_yearmonth_year', 'values': ['2022']}], 'sorts': [{'field_name': 'national_pension_category_summary.data_create_yearmonth_year', 'direction': 'asc'}, {'field_name': 'national_pension_category_summary.data_create_yearmonth_month', 'direction': 'asc'}], 'parameters': [], 'pivots': [], 'hidden_fields': []}


In [65]:
def make_dimension_and_description_pair(related_fields, schema_fields):
  dimension_and_description_pair = []
  for one_dimension in related_fields['dimensions']:
    for dimension in schema_fields:
      if dimension['field_name'] == one_dimension:
        dimension_and_description_pair.append((one_dimension, dimension['description']))
  return dimension_and_description_pair


In [66]:
def choose_chart_type_and_pivots(related_fields, schema_fields, question):
  dimension_and_description_pair = make_dimension_and_description_pair(related_fields, schema_fields)
  sample_json = """{
  "chart_type": "looker_column",
  "date_time_dimensions": ["dimension1"],
  "pivots": [
    "field1"
  ],
  "hidden_fields": [
    "field1"
  ]
  "reason_to_choose": "I choose field1 as a pivot field because ..."
  }"""
  prompt_template = """As a looker developer, choose chart type and pivot fields and hidden fields in the given dimensions for the question below. 
  Pivot field is a field that is used to create a pivot table. A pivot field converts category values in the field to columns so that you can compare different category values. 
  For example, if you have sales data, you can compare sales by product by setting the "Product" field as a pivot field. Date/time fields MUST not be used as pivot fields.
  Hidden field is a field that is not displayed in a chart. Hidden fields are used to hide fields that are not needed in the chart or that can confuse users. 
  For example, the "Product ID" field can be used to identify products, but it does not need to be displayed in a chart. If there are two same date fields, one of them can be hidden. 
  At least one field must be a visible field that is not used in pivot fields or hidden fields.

  chart_types : 
  looker_column - Column charts are useful when you want to compare the values of multiple fields for multiple records. It needs one main field to show the values separated by the main field. And this field must not be a pivot field.
  looker_line - Line charts are useful when you want to show the changes in a value over time. They are also useful for comparing the changes in two or more values over time.
  looker_area - Area charts are useful when you want to compare the trends of two or more values over time. They are also useful for showing the cumulative sum of values over time.
  looker_funnel - Funnel charts are useful to understand events in a sequential process, like prospect stages in a sales pipeline, engagement with a marketing campaign, or visitor movement through a website.
  looker_pie - Pie charts are useful when you want to show the proportion of values to the total value. They are also useful for comparing the proportional differences between values.
  looker_timeline - Timeline charts are useful when you want to show events over time. They are also useful for showing the duration of events. It needs at least 3 fields. 1. Event Name 2. Start Date 3. End Date
  looker_table - Table charts are useful when you want to show the values of multiple fields for multiple records. They are also useful for showing the values of multiple fields for a single record.

  dimensions : 
  {dimensions}

  question:
  {question}

  answer format: json
  {sample_json}
  """
  prompt_full = prompt_template.format(dimensions=dimension_and_description_pair, question=question, sample_json=sample_json)
  #print(prompt_full)
  response = llm.predict(prompt_full)
  return response

In [67]:
from langchain.pydantic_v1 import BaseModel, Field, validator
from typing import List

class LookerChartTypeClassifier(BaseModel):
  chart_type: str = Field(description="chart_type")
  date_time_dimensions: List[str] = Field(description="date_time_dimensions")
  pivots: List[str] = Field(description="pivot fields")
  hidden_fields: List[str] = Field(description="hidden fields")
  reason_to_choose: str = Field(description="reason_to_choose")

def get_looker_chart_type(reponse):
  parser = PydanticOutputParser(pydantic_object=LookerChartTypeClassifier)
  return parser.parse(response)


In [68]:
llm_response = choose_chart_type_and_pivots(related_fields, schema_fields, question)

In [69]:
# def parse_valid_json_values(response):
#   print('llm response:'+ response)
#   start_index = response.find('{')
#   end_index = response.rfind('}')
#   json_data = response[start_index:end_index+1]
#   parsed_json = json.loads(json_data)
#   return parsed_json

chart_type_and_pivots = parse_json_response(llm_response)

In [70]:
# chart_type_and_pivots = get_looker_chart_type(llm_response)
chart_type_and_pivots

{'chart_type': 'looker_column',
 'date_time_dimensions': ['national_pension_category_summary.data_create_yearmonth_month'],
 'pivots': ['bizcategory.cat_l1'],
 'hidden_fields': ['national_pension_category_summary.data_create_yearmonth_year'],
 'reason_to_choose': 'I choose national_pension_category_summary.data_create_yearmonth_month as a date_time_dimension because it is the most granular date field available. I choose bizcategory.cat_l1 as a pivot field because it is the most relevant field to the question. I choose to hide national_pension_category_summary.data_create_yearmonth_year because it is not relevant to the question.'}

In [71]:
def validate_hidden_fields(chart_type_and_pivots, related_fields):
  hidden_fields = chart_type_and_pivots['hidden_fields']
  pivot_fields = chart_type_and_pivots['pivots']
  dimensions = related_fields['dimensions']
  if len(dimensions) == len(hidden_fields) + len(pivot_fields):
    return False
  return True

def refine_hidden_fields(chart_type_and_pivots, question):
  hidden_fields = chart_type_and_pivots['hidden_fields']
  pivot_fields = chart_type_and_pivots['pivots']
  dimensions = hidden_fields
  
  sample_json = """{
  "dimension": "field1"
  }"""

  prompt_template = """As a looker developer, choose an appropriate field shown in the x-axis with the given dimensions and question.

output json format:
{sample_json}

given dimensions :
{dimensions}

given question:
{question}

  """
  prompt_full = prompt_template.format(sample_json=sample_json, dimensions=dimensions, question=question)
  response = llm.predict(prompt_full)
  return response


In [72]:
if not validate_hidden_fields(chart_type_and_pivots, related_fields): 
  field_shown = parse_json_response(refine_hidden_fields(chart_type_and_pivots, question))
  chart_type_and_pivots['hidden_fields'].remove(field_shown['dimension'])

In [73]:
chart_type_and_pivots['hidden_fields']

['national_pension_category_summary.data_create_yearmonth_year']

In [74]:
def make_query_for_the_look(lookml_model, lookml_explore, related_fields, valid_filter_values):
  fields = []
  fields.extend(related_fields['dimensions'])
  fields.extend(related_fields['measures'])
  filters = get_lookml_filter_values(valid_filter_values, related_fields['filters'])
  hidden_fields = chart_type_and_pivots['hidden_fields']
  pivot_fields = chart_type_and_pivots['pivots']
  chart_type = chart_type_and_pivots['chart_type']
  print('fields:' + str(fields))
  print('filters:' + str(filters))
  print('hidden_fields:' + str(hidden_fields))
  print('pivot_fields:' + str(pivot_fields))
  print('chart_type:' + str(chart_type))
  query_template = ml.WriteQuery(model=lookml_model, view=lookml_explore,fields=fields,filters=filters,pivots=pivot_fields,vis_config={'type':chart_type, 'hidden_fields':hidden_fields})
  query = sdk.create_query(query_template)
  run_response = sdk.run_inline_query("json", query)
  print('query.id:' + str(query.id))
  return query
    

In [75]:
lookml_filter = get_lookml_filter_values(valid_filter_values, related_fields['filters'])

['2022']


In [81]:
related_fields

{'dimensions': ['national_pension_category_summary.data_create_yearmonth_year',
  'national_pension_category_summary.data_create_yearmonth_month',
  'bizcategory.cat_l1'],
 'measures': ['national_pension_category_summary.avg_of_monthly_fixed_amount'],
 'filters': [{'field_name': 'bizcategory.cat_l1', 'values': ['금융 및 보험업']},
  {'field_name': 'national_pension_category_summary.data_create_yearmonth_year',
   'values': ['2022']}],
 'sorts': [{'field_name': 'national_pension_category_summary.data_create_yearmonth_year',
   'direction': 'asc'},
  {'field_name': 'national_pension_category_summary.data_create_yearmonth_month',
   'direction': 'asc'}],
 'parameters': [],
 'pivots': [],
 'hidden_fields': []}

In [77]:
generated_query = make_query_for_the_look(lookml_model, lookml_explore, related_fields, valid_filter_values)
print(generated_query.id)



['2022']
fields:['national_pension_category_summary.data_create_yearmonth_year', 'national_pension_category_summary.data_create_yearmonth_month', 'bizcategory.cat_l1', 'national_pension_category_summary.avg_of_monthly_fixed_amount']
filters:{'bizcategory.cat_l1': '금융 및 보험업', 'national_pension_category_summary.data_create_yearmonth_year': '2022'}
hidden_fields:['national_pension_category_summary.data_create_yearmonth_year']
pivot_fields:['bizcategory.cat_l1']
chart_type:looker_column
query.id:1383
1383


In [78]:
## Caution : 'title' field should be unique for each look. You should change this field to create new look.

def create_look(question, generated_query):
  existing_look = sdk.search_looks(query_id=generated_query.id)
  if len(existing_look) > 0:
    return existing_look[0]
  new_look = sdk.create_look(ml.WriteLookWithQuery(query_id=generated_query.id, 
    description=question,
    deleted=False,
    is_run_on_load=True,
    public=True,
    folder_id=str(sdk.me().personal_folder_id),
    title=question))
  return new_look


In [79]:
new_look = create_look(question + "1", generated_query)

In [ ]:
def get_full_url(short_url):
  return 'http://localhost:8080' + short_url

print(get_full_url(new_look.short_url))

In [ ]:
for group in sdk.all_groups():
  print(group)

In [ ]:
sso_url = sdk.create_sso_embed_url(ml.EmbedSsoParams(target_url=get_full_url(new_look.short_url), group_ids=[3]))

In [ ]:
print(sso_url.url)

In [ ]:
for one_look in sdk.all_looks():
  print(one_look.title)
  query = sdk.query(one_look.query_id)
  print(query)


THe below image is a look created by the above codes. 

![alt Looker View image](resources/looker_view_9.png "Title")

![alt Looker View image](resources/looker_view_8.png "Title")

In [86]:
test_json_array = json.loads("""[{"national_pension_mom.data_create_yearmonth_month":"2022-10"},
{"national_pension_mom.data_create_yearmonth_month":"2022-09"},
{"national_pension_mom.data_create_yearmonth_month":"2022-08"},
{"national_pension_mom.data_create_yearmonth_month":"2022-07"},
{"national_pension_mom.data_create_yearmonth_month":"2022-06"},
{"national_pension_mom.data_create_yearmonth_month":"2022-05"},
{"national_pension_mom.data_create_yearmonth_month":"2022-04"},
{"national_pension_mom.data_create_yearmonth_month":"2022-03"},
{"national_pension_mom.data_create_yearmonth_month":"2022-02"},
{"national_pension_mom.data_create_yearmonth_month":"2022-01"}]""")

def get_value_list_from_json_array(json_array):
  values = []
  for json_object in json_array:
    values.append(list(json_object.values())[0])
  return values

print(get_value_list_from_json_array(test_json_array))

['2022-10', '2022-09', '2022-08', '2022-07', '2022-06', '2022-05', '2022-04', '2022-03', '2022-02', '2022-01']
